In [219]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

# cities = gpd.read_feather(
#     "/Users/test/Documents/code/IL2/factory_loc_service/examples/closest_cities.feather"
# )
cities = gpd.read_parquet(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/cities.parquet"
)

uinput_industry = "oil_and_gas_ext"
plant_attr_coef = 0.1
uinput_spec_num = {"Инженер-конструктор": 200, "Инженер-проектировщик": 300}
# city_names = cities["region_city"].values
city_names = ["Томская область, Томск"]


In [220]:
# print(uinput_industry, uinput_spec_num, city_names)


In [252]:
ontology = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/new_ontology.pkl"
)

grouped_grads = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/grouped_grads.pkl"
)

cv = pd.read_parquet("../api/app/data/cv.gzip").rename(
    columns={"hh_name": "speciality"}
)
YEAR = 2021
cv = cv[cv["year"] == YEAR]


In [254]:
ontology['prof_domain'].unique()


array(['Добыча нефти и газа', 'Добыча угля и металлических руд',
       'Машиностроение', 'Судостроение',
       'Авиастроение и космическая отрасль', 'Цветная металлургия',
       'Черная металлургия', 'Химическая промышленность', 'Фармацевтика',
       'Электроника и микроэлектроника'], dtype=object)

In [222]:
ontology["speciality"].unique()


array(['Оператор, аппаратчик', 'Монтажник', 'Машинист',
       'Мастер по ремонту оборудования', 'Разнорабочий', 'Слесарь',
       'Геодезист', 'Геолог', 'Технолог', 'Инженер-эколог',
       'Инженер-конструктор', 'Сварщик', 'Наладчик',
       'Токарь, фрезеровщик, шлифовщик', 'Контролер ОТК',
       'Инженер-проектировщик', 'Специалист по сертификации',
       'Инженер по качеству', 'Механик', 'Инженер по эксплуатации',
       'Лаборант', 'Исследователь', 'Электромонтажник'], dtype=object)

In [223]:
mask = grouped_grads.reset_index(drop=False).loc[:, "cluster_center"].isin(city_names)
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .loc[mask, :]
    .set_index(["cluster_center", "region_city", "type", "edu_group_code"])
)

mask = cv["region_city"].isin(city_names)
cv = cv[mask]


In [224]:
# ontology = pd.read_csv(
#     "/Users/test/Documents/code/IL2/industrial-location/api/app/data/ontology.csv",
#     index_col=0,
# )

# ontology.loc[ontology["industry_code"] == "mining_oil_gas", "industry_code"] = (
#     "oil_and_gas_ext"
# )
# ontology.loc[ontology["industry_code"] == "mmining_coal_ores", "industry_code"] = (
#     "coal_and_metal_ext"
# )
# ontology.loc[ontology["industry_code"] == "mech_engineering", "industry_code"] = (
#     "machinery"
# )
# ontology.loc[ontology["industry_code"] == "shipbuilding", "industry_code"] = (
#     "shipbuilding"
# )
# ontology.loc[ontology["industry_code"] == "aircraft_engineering", "industry_code"] = (
#     "aircraft_and_space"
# )
# ontology.loc[ontology["industry_code"] == "non_ferrous_metallurgy", "industry_code"] = (
#     "nonferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "ferrous_metallurgy", "industry_code"] = (
#     "ferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "chemical", "industry_code"] = "chemicals"
# ontology.loc[ontology["industry_code"] == "pharma", "industry_code"] = "pharmacy"
# ontology.loc[ontology["industry_code"] == "electronics", "industry_code"] = (
#     "electronics"
# )


In [225]:
# ontology["industry_code"].unique()


In [226]:
# ontology.to_pickle("new_ontology.pkl")


In [227]:
cv = cv.merge(ontology[["speciality", "edu_group_code"]], on="speciality")
cv["type"] = "CV"
grouped_cv = (
    cv.groupby(["cluster_center", "region_city", "type", "edu_group_code"])["id_cv"]
    .count()
    .to_frame()
)
cv.head(3)


,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,4e4f2a00-c1e3-11eb-a1ad-3bfa22f2d66b,Геолог,2021,"Томская область, Томск","Томская область, Томск",21,CV
1,4e4f2a00-c1e3-11eb-a1ad-3bfa22f2d66b,Геолог,2021,"Томская область, Томск","Томская область, Томск",21,CV
2,4e4f2a00-c1e3-11eb-a1ad-3bfa22f2d66b,Геолог,2021,"Томская область, Томск","Томская область, Томск",5,CV


In [228]:
grads = pd.read_csv("graduates.csv")
grads.dropna(subset="edu_group_code", inplace=True)
grouped_grads = (
    grads.groupby(["cluster_center", "region_city", "type", "edu_group_code"])[
        "graduates_per_year_forecast"
    ]
    .sum()
    .to_frame()
)
grouped_grads = grouped_grads.join(grouped_cv, how="outer")
grouped_grads["city_capacity_grads_and_cv_sum"] = grouped_grads[
    "graduates_per_year_forecast"
].fillna(0) + grouped_grads["id_cv"].fillna(0)
grouped_grads.to_pickle("grouped_grads.pkl")


In [229]:
grouped_grads


graduates_per_year_forecast  id_cv  city_capacity_grads_and_cv_sum
cluster_center                 region_city                    type edu_group_code                                                                    
Адыгея, Майкоп                 Адыгея, Майкоп                 ВПО  1                                     494.0    NaN                           494.0
                                                                   2                                     229.0    NaN                           229.0
                                                                   3                                      45.0    NaN                            45.0
                                                                   4                                     103.0    NaN                           103.0
                                                                   6                                     356.0    NaN                           356.0
...                                                                                                        ...    ...                             ...
Ярославская область, Ярославль Ярославская область, Ярославль СПО  46                                     54.0    NaN                            54.0
                                                                   49                                    175.0    NaN                           175.0
                                                                   51                                     20.0    NaN                            20.0
                                                                   53                                     63.0    NaN                            63.0
                                                                   54                                    203.0    NaN                           203.0

[15509 rows x 3 columns]

In [230]:
# pd.read_csv("graduates.csv")


In [231]:
pd.read_csv("graduates.csv")["edu_group_code"].unique()


array([ 9, 10, 19, 38, 40, 42, 43, 51, 53, 54,  8, 21, 23, 15, 35, 36, 13,
       29, 18, 27, 44, 49, 39, 11, 22, 46, 31, 34, 33, 20, 32,  7, 52, 55,
       24, 12, 14, 26, 25,  5, 50, 37, 41, 45, 47, 48,  1,  2, 28, 58,  3,
        4,  6, 16, 30, 56, 17])

In [232]:
print(grouped_grads.reset_index()["edu_group_code"].unique())


[ 1  2  3  4  6  8  9 10 13 15 18 19 20 21 23 27 29 31 33 35 36 37 38 39
 40 42 43 44 45 46 49 52 54  7 34 53  5 51 11 12 16 22 32 41 47 50 17 24
 26 28 30 14 48 25 58 55 56]


In [233]:
# --- START ---
"""
Эта штука завязана на входных профессиях и индустрии,
поэтому нужно каждый раз пересчитывать -- мб можно оптимизировать и пересчитывать только часть
но имхо оно и так быстро
"""

uinput_spec_num_2 = dict()
competitor_industries = list()
competitor_fatories = list()


for k in uinput_spec_num.keys():
    uinput_spec_num_2[k] = (
        ontology.loc[ontology["speciality"] == k, "edu_group_id"]
        .drop_duplicates()
        .values.tolist()
    )
    competitor_industries += (
        ontology.loc[ontology["speciality"] == k, "industry_code"]
        .drop_duplicates()
        .tolist()
    )
# --- START ---

competitor_industries = set(competitor_industries)
# competitor_industries.remove(uinput_industry)


grad_col = []
fact_col = []

for col in cities.columns:
    if "factor" in col:
        fact_col.append(col)

    if "factor" in col and any(industry in col for industry in competitor_industries):
        competitor_fatories.append(col)

    if "grad" in col:
        grad_col.append(col)

cities["competitors_factories_num"] = cities[competitor_fatories].sum(axis=1)

print(competitor_industries)
print(grad_col, fact_col, competitor_fatories)


{'shipbuilding', 'electronics', 'aircraft_and_space', 'machinery'}
[] ['factories_total'] []


In [234]:
grouped_grads = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/grouped_grads.pkl"
)
grouped_grads


graduates_per_year_forecast  id_cv  city_capacity_grads_and_cv_sum
cluster_center                 region_city                    type edu_group_code                                                                    
Адыгея, Адыгейск               Адыгея, Адыгейск               CV   5                                       NaN    2.0                             2.0
                                                                   11                                      NaN    3.0                             3.0
                                                                   15                                      NaN   21.0                            21.0
                                                                   18                                      NaN   12.0                            12.0
                                                                   19                                      NaN    2.0                             2.0
...                                                                                                        ...    ...                             ...
Ярославская область, Ярославль Ярославская область, Ярославль СПО  46                                     54.0    NaN                            54.0
                                                                   49                                    175.0    NaN                           175.0
                                                                   51                                     20.0    NaN                            20.0
                                                                   53                                     63.0    NaN                            63.0
                                                                   54                                    203.0    NaN                           203.0

[79890 rows x 3 columns]

In [235]:
uinput_spec_num_2


{'Инженер-конструктор': [3, 9, 21], 'Инженер-проектировщик': [4, 8, 10]}

In [236]:
groups = []

for k, v in uinput_spec_num_2.items():
    groups += v


In [237]:
groups = set(groups)


In [238]:
groups


{3, 4, 8, 9, 10, 21}

In [239]:
cv.drop_duplicates(subset=["speciality", "edu_group_code"])


,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,4e4f2a00-c1e3-11eb-a1ad-3bfa22f2d66b,Геолог,2021,"Томская область, Томск","Томская область, Томск",21,CV
2,4e4f2a00-c1e3-11eb-a1ad-3bfa22f2d66b,Геолог,2021,"Томская область, Томск","Томская область, Томск",5,CV
6,13095360-304f-11eb-9502-736ab11edb0c,Инженер-конструктор,2021,"Томская область, Томск","Томская область, Томск",15,CV
7,13095360-304f-11eb-9502-736ab11edb0c,Инженер-конструктор,2021,"Томская область, Томск","Томская область, Томск",24,CV
8,13095360-304f-11eb-9502-736ab11edb0c,Инженер-конструктор,2021,"Томская область, Томск","Томская область, Томск",11,CV
...,...,...,...,...,...,...,...
236,67b09060-b932-11eb-90d0-ab5d2eb93a75,Лаборант,2021,"Томская область, Томск","Томская область, Томск",21,CV
238,67b09060-b932-11eb-90d0-ab5d2eb93a75,Лаборант,2021,"Томская область, Томск","Томская область, Томск",19,CV
575,8060e5b0-9858-11eb-912b-3bfa22f2d66b,Контролер ОТК,2021,"Томская область, Томск","Томская область, Томск",15,CV
578,8060e5b0-9858-11eb-912b-3bfa22f2d66b,Контролер ОТК,2021,"Томская область, Томск","Томская область, Томск",22,CV


In [240]:
print(grouped_grads.reset_index()["edu_group_code"].unique())


[ 5 11 15 18 19 21 22 24 26 33  4  6  8  1  2  3  9 10 13 20 23 27 29 31
 35 36 37 38 39 40 42 43 44 45 46 49 52 54  7 34 53 51 12 16 32 41 47 50
 17 28 30 14 48 25 58 55 56]


In [241]:
grouped_grads.reset_index(drop=False, inplace=True)
mask_groups = grouped_grads["edu_group_code"].isin(groups)
grouped_grads = grouped_grads[mask_groups]


In [242]:
grouped_grads


,cluster_center,region_city,type,edu_group_code,graduates_per_year_forecast,id_cv,city_capacity_grads_and_cv_sum
5,"Адыгея, Адыгейск","Адыгея, Адыгейск",CV,21,NaN,14.0,14.0
14,"Адыгея, Адыгейск","Адыгея, Ассоколай",CV,21,NaN,2.0,2.0
19,"Адыгея, Адыгейск","Адыгея, Вочепший",CV,4,NaN,1.0,1.0
24,"Адыгея, Адыгейск","Адыгея, Вочепший",CV,21,NaN,4.0,4.0
29,"Адыгея, Адыгейск","Адыгея, Понежукай",CV,8,NaN,1.0,1.0
...,...,...,...,...,...,...,...
79838,"Ярославская область, Ярославль","Ярославская область, Ярославль",ВПО,21,12.0,NaN,12.0
79861,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,8,266.0,NaN,266.0
79862,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,9,618.0,NaN,618.0
79863,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,10,107.0,NaN,107.0


In [243]:
grouped_grads.loc[grouped_grads["type"] == "ВПО"]


,cluster_center,region_city,type,edu_group_code,graduates_per_year_forecast,id_cv,city_capacity_grads_and_cv_sum
68,"Адыгея, Майкоп","Адыгея, Майкоп",ВПО,3,45.0,NaN,45.0
69,"Адыгея, Майкоп","Адыгея, Майкоп",ВПО,4,103.0,NaN,103.0
71,"Адыгея, Майкоп","Адыгея, Майкоп",ВПО,8,554.0,NaN,554.0
72,"Адыгея, Майкоп","Адыгея, Майкоп",ВПО,9,627.0,NaN,627.0
73,"Адыгея, Майкоп","Адыгея, Майкоп",ВПО,10,525.0,NaN,525.0
...,...,...,...,...,...,...,...
79826,"Ярославская область, Ярославль","Ярославская область, Ярославль",ВПО,4,924.0,NaN,924.0
79830,"Ярославская область, Ярославль","Ярославская область, Ярославль",ВПО,8,682.0,NaN,682.0
79831,"Ярославская область, Ярославль","Ярославская область, Ярославль",ВПО,9,1893.0,NaN,1893.0
79832,"Ярославская область, Ярославль","Ярославская область, Ярославль",ВПО,10,446.0,NaN,446.0


In [244]:
grouped_grads.loc[grouped_grads["type"] == "ВПО", "type"] = "СПО"
grouped_grads.loc[grouped_grads["type"] == "СПО", "type"] = "graduates"
grouped_grads.loc[grouped_grads["type"] == "CV", "type"] = "specialists"

grouped_grads.rename(columns={"id_cv": "cv_count"}, inplace=True)

grouped_grads = (
    grouped_grads.set_index("cluster_center")
    .groupby(["cluster_center", "type", "edu_group_code"])[
        ["city_capacity_grads_and_cv_sum", "graduates_per_year_forecast", "cv_count"]
    ]
    .sum()
    .join(
        cities[
            ["region_city", "population", "num_in_migration"]
            + grad_col
            + fact_col
            + ["factories_total"]
            + ["competitors_factories_num"]
        ]
        .rename(columns={"region_city": "cluster_center"})
        .set_index("cluster_center"),
        how="left",
    )
)

grouped_grads


/var/folders/jn/d6x3c7jd7594dtldxq22ysn80000gn/T/ipykernel_57823/4249309167.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_grads.rename(columns={"id_cv": "cv_count"}, inplace=True)


city_capacity_grads_and_cv_sum  graduates_per_year_forecast  cv_count  population  num_in_migration  factories_total  factories_total  competitors_factories_num
cluster_center                 type        edu_group_code                                                                                                                                                                  
Адыгея, Адыгейск               specialists 4                                          1.0                          0.0       1.0     13191.0          3.583744              0.0              0.0                        0.0
                                           8                                          1.0                          0.0       1.0     13191.0          3.583744              0.0              0.0                        0.0
                                           21                                        22.0                          0.0      22.0     13191.0          3.583744              0.0              0.0                        0.0
Адыгея, Майкоп                 graduates   3                                         45.0                         45.0       0.0    139687.0         20.385065              6.0              6.0                        0.0
                                           4                                        103.0                        103.0       0.0    139687.0         20.385065              6.0              6.0                        0.0
...                                                                                   ...                          ...       ...         ...               ...              ...              ...                        ...
Ярославская область, Ярославль graduates   10                                       553.0                        553.0       0.0    570824.0        442.679901             65.0             65.0                        0.0
                                           21                                       132.0                        132.0       0.0    570824.0        442.679901             65.0             65.0                        0.0
                               specialists 4                                         24.0                          0.0      24.0    570824.0        442.679901             65.0             65.0                        0.0
                                           8                                         16.0                          0.0      16.0    570824.0        442.679901             65.0             65.0                        0.0
                                           21                                       354.0                          0.0     354.0    570824.0        442.679901             65.0             65.0                        0.0

[4502 rows x 8 columns]

In [245]:
uinput_spec_num_2


{'Инженер-конструктор': [3, 9, 21], 'Инженер-проектировщик': [4, 8, 10]}

In [246]:
grouped_grads["working_population"] = (
    (grouped_grads["population"] * 0.65).round(0).astype(int)
)

grouped_grads.drop(columns=["population"], inplace=True)

grouped_grads = grouped_grads.drop(
    columns=[
        "city_capacity_grads_and_cv_sum",
        "num_in_migration",
        "working_population",
        "factories_total",
    ]
).rename(columns={"graduates_per_year_forecast": "grads"})


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [100]:
"""
Получается что миграции которые мы растим влияют только на кол-во резюме
Условимся что изначально у нас все находится в равновесии
Как только мы меняем что-то -- у нас кол-во резюме умножается на процент изменения входящей миграции (было 100 - стало 200 -- умножаем на 2)
"""


'\nПолучается что миграции которые мы растим влияют только на кол-во резюме\nУсловимся что изначально у нас все находится в равновесии\nКак только мы меняем что-то -- у нас кол-во резюме умножается на процент изменения входящей миграции (было 100 - стало 200 -- умножаем на 2)\n'

In [101]:
grouped_grads.loc[
    grouped_grads["competitors_factories_num"] == 0, "competitors_factories_num"
] = 1


In [102]:
grouped_grads["grads"] = (
    grouped_grads["grads"] / grouped_grads["competitors_factories_num"]
)
grouped_grads["cv_count"] = (
    grouped_grads["cv_count"] / grouped_grads["competitors_factories_num"]
)


In [103]:
grouped_grads.reset_index(inplace=True)


In [104]:
uinput_spec_num


{'Инженер-конструктор': 200, 'Инженер-проектировщик': 300}

In [105]:
uinput_spec_num_2


{'Инженер-конструктор': [3, 9, 21], 'Инженер-проектировщик': [4, 8, 10]}

In [106]:
plant_attr_coef


0.1

In [247]:
grouped_grads.reset_index(inplace=True)
grouped_grads = grouped_grads[grouped_grads["cluster_center"].isin(city_names)]


In [248]:
# Шаг 1: Создание нового столбца 'speciality' на основе словаря
def map_specialty(edu_code):
    for specialty, codes in uinput_spec_num_2.items():
        if edu_code in codes:
            return specialty
    return "Другое"  # Если не нашли специальность


grouped_grads["speciality"] = grouped_grads["edu_group_code"].apply(map_specialty)


/var/folders/jn/d6x3c7jd7594dtldxq22ysn80000gn/T/ipykernel_57823/2038659540.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_grads["speciality"] = grouped_grads["edu_group_code"].apply(map_specialty)


In [251]:
uinput_spec_num_2


{'Инженер-конструктор': [3, 9, 21], 'Инженер-проектировщик': [4, 8, 10]}

In [249]:
grouped_grads


,cluster_center,type,edu_group_code,city_capacity_grads_and_cv_sum,graduates_per_year_forecast,cv_count,population,num_in_migration,factories_total,factories_total,competitors_factories_num,speciality
3821,"Томская область, Томск",graduates,3,1560.0,1560.0,0.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-конструктор
3822,"Томская область, Томск",graduates,4,1081.0,1081.0,0.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-проектировщик
3823,"Томская область, Томск",graduates,8,769.0,769.0,0.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-проектировщик
3824,"Томская область, Томск",graduates,9,7849.0,7849.0,0.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-конструктор
3825,"Томская область, Томск",graduates,10,2168.0,2168.0,0.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-проектировщик
3826,"Томская область, Томск",graduates,21,1105.0,1105.0,0.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-конструктор
3827,"Томская область, Томск",specialists,4,26.0,0.0,26.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-проектировщик
3828,"Томская область, Томск",specialists,8,22.0,0.0,22.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-проектировщик
3829,"Томская область, Томск",specialists,21,523.0,0.0,523.0,551504.0,118.437355,32.0,32.0,0.0,Инженер-конструктор


In [250]:
# Шаг 2: Группировка по cluster_center и speciality для подсчета выпускников и специалистов
result = (
    grouped_grads.groupby(["cluster_center", "speciality"])
    .agg({"grads": "sum", "cv_count": "sum"})  # Сумма выпускников  # Сумма специалистов
    .reset_index()
)

# Шаг 3: Переименование столбцов для ясности
result.rename(
    columns={"grads": "total_graduates", "cv_count": "total_specialists"}, inplace=True
)


KeyError: "Column(s) ['grads'] do not exist"

In [109]:
grouped_grads


,cluster_center,type,edu_group_code,grads,cv_count,competitors_factories_num,speciality
0,"Краснодарский край, Тихорецк",graduates,8,146.0,0.0,1.0,Инженер-проектировщик
1,"Краснодарский край, Тихорецк",specialists,4,0.0,8.0,1.0,Инженер-проектировщик
2,"Краснодарский край, Тихорецк",specialists,8,0.0,17.0,1.0,Инженер-проектировщик
3,"Краснодарский край, Тихорецк",specialists,21,0.0,116.0,1.0,Инженер-конструктор
4,"Ростовская область, Азов",graduates,8,80.0,0.0,1.0,Инженер-проектировщик
5,"Ростовская область, Азов",specialists,4,0.0,5.0,1.0,Инженер-проектировщик
6,"Ростовская область, Азов",specialists,8,0.0,45.0,1.0,Инженер-проектировщик
7,"Ростовская область, Азов",specialists,21,0.0,262.0,1.0,Инженер-конструктор
8,"Ростовская область, Аксай",graduates,8,27.0,0.0,1.0,Инженер-проектировщик
9,"Ростовская область, Аксай",specialists,8,0.0,3.0,1.0,Инженер-проектировщик


In [110]:
uinput_spec_num


{'Инженер-конструктор': 200, 'Инженер-проектировщик': 300}

In [111]:
# Инициализируем колонки числовыми значениями
result["prov_graduates"] = pd.NA  # Используем Pandas NA
result["prov_specialists"] = pd.NA

# Проверяем ключи и значения
# print("Словарь uinput_spec_num:")
# print(uinput_spec_num)

# print("\nУникальные специальности:")
# print(result['speciality'].unique())

# Выполняем расчеты
for spec in result["speciality"].unique():
    if spec in uinput_spec_num:
        factor = uinput_spec_num[spec]
        # print(f"\nОбработка специальности '{spec}' с коэффициентом {factor}")

        # Логика расчета
        grads_values = (
            result.loc[result["speciality"] == spec, "total_graduates"].fillna(0)
            / factor
        )
        specialists_values = (
            result.loc[result["speciality"] == spec, "total_specialists"].fillna(0)
            / factor
        )

        # Устанавливаем значения напрямую
        result.loc[result["speciality"] == spec, "prov_graduates"] = round(
            grads_values, 2
        )
        result.loc[result["speciality"] == spec, "prov_specialists"] = round(
            specialists_values, 2
        )
    else:
        print(f"Специальность '{spec}' не найдена в словаре.")

# Проверяем итоговый результат
# print("\nИтоговый DataFrame:")
# print(result[['speciality', 'total_graduates', 'total_specialists', 'prov_graduates', 'prov_specialists']])


In [ ]:
import numpy as np

# Проверяем и обновляем значения в 'prov_graduates'
result["prov_graduates"] = np.where(
    result["prov_graduates"] > 1, 1, result["prov_graduates"]
)

# Проверяем и обновляем значения в 'prov_specialists'
result["prov_specialists"] = np.where(
    result["prov_specialists"] > 1, 1, result["prov_specialists"]
)


In [38]:
result


,cluster_center,speciality,total_graduates,total_specialists,prov_graduates,prov_specialists
0,"Краснодарский край, Тихорецк",Инженер-конструктор,0.0,116.0,0.0,0.58
1,"Краснодарский край, Тихорецк",Инженер-проектировщик,146.0,25.0,0.49,0.08
2,"Ростовская область, Азов",Инженер-конструктор,0.0,262.0,0.0,1
3,"Ростовская область, Азов",Инженер-проектировщик,80.0,50.0,0.27,0.17
4,"Ростовская область, Аксай",Инженер-конструктор,0.0,40.0,0.0,0.2
5,"Ростовская область, Аксай",Инженер-проектировщик,27.0,3.0,0.09,0.01
6,"Ростовская область, Батайск",Инженер-конструктор,0.0,90.0,0.0,0.45
7,"Ростовская область, Батайск",Инженер-проектировщик,47.0,5.0,0.16,0.02
8,"Ростовская область, Белая Калитва",Инженер-конструктор,0.0,118.0,0.0,0.59
9,"Ростовская область, Белая Калитва",Инженер-проектировщик,64.0,21.0,0.21,0.07


In [324]:
"""
(!) Сейчас есть всё кроме непосредственной оценки (!)
А ну и задаваемое кол-во специалистов пока не учитывается
Но это следствие из отсутствия реализации расчета оценки

Эту таблицу практически в таком виде и планирую отдавать

Из того что обсуждали с СА:
условно выпускников взвесить по кол-ву предприятий тк их число ту мач большое
"""


'\n(!) Сейчас есть всё кроме непосредственной оценки (!)\nА ну и задаваемое кол-во специалистов пока не учитывается\nНо это следствие из отсутствия реализации расчета оценки\n\nЭту таблицу практически в таком виде и планирую отдавать\n\nИз того что обсуждали с СА:\nусловно выпускников взвесить по кол-ву предприятий тк их число ту мач большое\n'

In [325]:
# grouped_grads


grads  cv_count  competitors_factories_num
cluster_center                         type        edu_group_code                                             
Краснодарский край, Тихорецк           graduates   11                88.0       0.0                        1.0
                                                   15                63.0       0.0                        1.0
                                       specialists 6                  0.0       8.0                        1.0
                                                   11                 0.0      43.0                        1.0
                                                   15                 0.0     407.0                        1.0
Ростовская область, Азов               graduates   15               280.0       0.0                        1.0
                                       specialists 6                  0.0       5.0                        1.0
                                                   11                 0.0     105.0                        1.0
                                                   15                 0.0     551.0                        1.0
Ростовская область, Аксай              graduates   11                26.0       0.0                        1.0
                                                   15                17.0       0.0                        1.0
                                       specialists 11                 0.0      15.0                        1.0
                                                   15                 0.0     117.0                        1.0
Ростовская область, Батайск            graduates   11                80.0       0.0                        1.0
                                                   15                30.0       0.0                        1.0
                                       specialists 6                  0.0       1.0                        1.0
                                                   11                 0.0      41.0                        1.0
                                                   15                 0.0     219.0                        1.0
Ростовская область, Белая Калитва      graduates   15                68.0       0.0                        1.0
                                       specialists 6                  0.0       4.0                        1.0
                                                   11                 0.0      47.0                        1.0
                                                   15                 0.0     160.0                        1.0
Ростовская область, Зверево            graduates   15                18.0       0.0                        1.0
                                       specialists 6                  0.0       3.0                        1.0
                                                   11                 0.0      12.0                        1.0
                                                   15                 0.0      41.0                        1.0
Ростовская область, Каменск-Шахтинский graduates   15               235.0       0.0                        1.0
                                       specialists 6                  0.0       6.0                        1.0
                                                   11                 0.0      41.0                        1.0
                                                   15                 0.0     194.0                        1.0
Ростовская область, Новочеркасск       graduates   11               765.0       0.0                        1.0
                                                   15              2024.0       0.0                        1.0
                                       specialists 6                  0.0      10.0                        1.0
                                                   11                 0.0     132.0                        1.0
                                                   15                 0.

In [326]:
""" ------------------------- FINISH HERE -------------------------------- """


' ------------------------- FINISH HERE -------------------------------- '